<a href="https://colab.research.google.com/github/sasuraibito1125/google_colab/blob/main/%E6%A8%99%E6%BA%96%E6%AD%A3%E8%A6%8F%E5%88%86%E5%B8%83%E8%A1%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 関数定義

In [1]:
# @title #### 四捨五入関数 { vertical-output: true, display-mode: "form" }
#@markdown 関数名: `round_up`
#@markdown * 引数：`precision=4`, `digit=None`
#@markdown * 戻り値：lambda function
def round_up(precision=4, digit=None):
  '''Generate lambda function to round off.

  ex)
  df.map(round_up(digit='0.0001'))
  df.map(round_up(digit='1E1'))

  Parameters
  ----------
  precision: precision of rounding. default is 4.
  digit: string of rounding digit. default is None(no used) and prioritize this to precision.

  Returns
  -------
  lambda function
    lambda function to round off.
  '''
  from decimal import Decimal, ROUND_HALF_UP
  def to_digit(precision):
    if precision == 0:
      return '0'
    else:
      return '0.' + ('0' * (precision - 1)) + '1'
  return lambda x: float(Decimal(str(x))
                            .quantize(Decimal(digit if digit else to_digit(precision)),
                                      rounding=ROUND_HALF_UP))

In [2]:
# @title #### 標準正規分布表生成関数 { vertical-output: true, display-mode: "form" }
# @markdown 関数名：`generate_snd_table`
# @markdown * 引数：`rows=None`, `columns=None`, `both=False`, `precision=5`
# @markdown * 戻り値：`DataFrame`
# @markdown * 依存関数：`round_up`
def generate_snd_table(rows=None, columns=None, both=False, precision=5):
  '''To generate the table of standard normal distribution.

  Parameters
  ----------
  rows: list of integer and 1st decimal place of the z-values.
    if undefined, the values are from 0 to 30 in increments of 0.1.
  columns: list of 2nd and 3rd decimal places of the z-values.
    if undefined, the values are from 0.00 to 0.09 in increments of 0.01.
  both: flag to specify one-sided or both-sided. default False(both-sided).
  precision: precision of the value of probability. default 5.

  Returns
  -------
  DataFrame
    table of standard normal distribution
  '''
  import numpy as np
  import pandas as pd
  from scipy.stats import norm

  def generate_default_columns():
    return np.array([i * 0.01 for i in range(10)])
  def generate_default_rows():
    return np.array([i * 0.1 for i in range(31)]).reshape(-1, 1)

  if not columns:
    columns = generate_default_columns()

  if not rows:
    rows = generate_default_rows()
  else:
    rows = np.array(rows)
    if rows.ndim == 1:
      rows = rows.reshape(-1, 1)
    elif rows.ndim == 2 and rows.shape[1] == 1:
      pass
    else:
      print('不正な自由度のリストが指定されました。デフォルト値を使用します。')
      rows = generate_default_rows()

  df = pd.DataFrame(norm.sf(columns + rows, loc=0, scale=1),
                      columns=list(map(lambda x: "%.2f" % x, columns)),
                      index=list(map(lambda x: "%.1f" % x, rows)))
  return (df * (2 if both else 1)).applymap(round_up(precision=precision))


In [3]:
# @title #### 表の行と列を強調する関数 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`highlight_subject`
#@markdown * 引数：`df`, `row=None`, `col=None`, `row_color='#FFD0FF'`, `col_color='#D1FDFF'`, `cross_color='#C7B5FF'`
#@markdown * 戻り値：`Styler`
def highlight_subject(df, row=None, col=None,
                      row_color='#FFD0FF',
                      col_color='#D1FDFF',
                      cross_color='#C7B5FF'):
  '''highlight row, column and cross point of DataFrame.

  Parameters
  ----------
  df: a target pandas DataFrame to be highlighted
  row: a target row to be highlighted
  col: a target column to be highlighted
  row_color: a highlight color of row
  col_color: a highlight color of column
  cross_color: a highlight color of cross point of highligh row and column

  Returns
  -------
  Styler
    pandas Styler object to be highlighted
  '''
  import pandas as pd

  idx = pd.IndexSlice
  styler = df.style
  if row:
    styler = styler.set_properties(**{'color':'black', 'background-color': row_color},
                                   subset=idx[idx[row], :])
  if col:
    styler = styler.set_properties(**{'color':'black', 'background-color': col_color},
                                   subset=idx[col])
  if col and row:
    styler = styler.set_properties(**{'color':'black', 'background-color': cross_color},
                                   subset=idx[idx[row], idx[col]])
  return styler


In [4]:
#@title #### 標準正規分布表から指定された $z$ 値に対する確率を取得する関数
#@markdown 関数名：`find_p_from_z_in_snd_table`
#@markdown * 引数：`snd`, `z`
#@markdown * 戻り値：`tuple<float, string, string>`
def find_p_from_z_in_snd_table(snd, z):
  '''Find the probability value from z-value in SND table.

  Parameters
  ----------
  snd: pandas DataFrame of SND table.
  z: target z-value.

  Returns
  -------
  tuple
    the probalibity value(float), row name(string), and column name(string)

  '''
  v=f'{abs(z):.02f}'
  row = v[:3]
  col = f'0.0{v[-1]}'
  return snd.at[row, col], row, col

In [5]:
# @title #### 標準正規分布表で指定された $z$ 値に対する行と列を強調表示する関数 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`highlight_z_in_snd_table`
#@markdown * 引数：`snd`, `z`
#@markdown * 戻り値：`Styler`
#@markdown * 依存関数：`find_p_from_z_in_snd_table`, `highlight_subject`
def highlight_z_in_snd_table(snd, z):
  '''Hightlight target z-value in SND table.

  Parameters
  ----------
  snd: pandas DataFrame of SND table.
  z: z-value of SND.

  Returns
  -------
  Styler
    pandas Styler object to be highlighted

  '''
  p = find_p_from_z_in_snd_table(snd, z)
  return highlight_subject(snd, row=p[1], col=p[2])

In [6]:
# @title #### 標準正規分布表から指定された確率に最も近い $z$ 値を取得する関数 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`find_z_from_p_in_snd_table`
#@markdown * 引数：`snd`, `p`
#@markdown * 戻り値：`tuple<float, string, string>`
def find_z_from_p_in_snd_table(snd, p):
  '''Find z-value of the nearest probability value in SND table.

  Parameters
  ----------
  snd: pandas DataFrame of SND table.
  p: target probability value.

  Returns
  -------
  tuple
    z-value(float), row name(string), and column name(string)

  '''
  import numpy as np

  def to_pair(snd, idx):
    return snd.index[idx // 10], snd.columns[idx % 10]

  data = np.abs(snd.to_numpy() - p)
  idx = data.argmin()
  # min_idxs = np.where(data == data.min())[0]
  # if len(min_idxs) != 1:
  #   print('複数:', min_idxs, idx)
  #   print('行/列:', [r + c[-1] for r, c in [to_pair(snd, i) for i in min_idxs]])
  row, col = to_pair(snd, idx)
  z = row + col[-1]
  return float(z), row, col

In [7]:
# @title #### 標準正規分布表で指定された確率に最も近い $z$ 値に対する行と列を強調表示する関数 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`highlight_probability_in_snd_table`
#@markdown * 引数：`snd`, `p`
#@markdown * 戻り値：`Styler`
#@markdown * 依存関数：`find_z_from_p_in_snd_table`, `highlight_z_in_snd_table`
def highlight_probability_in_snd_table(snd, p):
  '''Hightlight target probability value in SND table.

  Parameters
  ----------
  p: probability to find in SND table.

  Returns
  -------
  Styler
    pandas Styler object to be highlighted

  '''
  import numpy as np

  z = find_z_from_p_in_snd_table(snd, p)
  return highlight_z_in_snd_table(snd, z[0])

In [8]:
# @title #### 信頼係数から確率に変換する関数 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`cf2p`
#@markdown * 引数：`cf`
#@markdown * 戻り値：`float`
#@markdown
#@markdown $p = \dfrac{1 - \dfrac{CF}{100}}{2} = \dfrac{100 - CF}{200}$
def cf2p(cf):
  '''Convert confidence factor to probability.

  Parameters
  ----------
  cf: confidence factor

  Returns
  -------
  float
    probability
  '''
  return (100 - cf) / 200

In [9]:
#@title #### 標準正規分布表から指定された信頼係数となる $z$ 値を取得する関数 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`find_z_from_cf_in_snd_table`
#@markdown * 引数：`snd`, `cf=95`
#@markdown * 戻り値： `tuple<float, float, float, float>`
#@markdown * 依存関数：`find_z_from_p_in_snd_table`
def find_z_from_cf_in_snd_table(snd, cf=95):
  '''Find z-value of the nearest probability value in SND table.

  Parameters
  ----------
  snd: pandas DataFrame of SND table.
  cf: Target confidence factor. Default value is 95.

  Returns
  -------
  tuple
    z-value(float), probability, row name(string), and column name(string)
  '''
  p = cf2p(cf)
  z = find_z_from_p_in_snd_table(snd, p)
  return z[0], p, *z[1:]


In [10]:
#@title #### 母平均の信頼区間の計算 { display-mode: "form" }
#@markdown 関数名：`calc_conf_interval_snd`
#@markdown * 引数：`xbar`, `s`, `n`, `cf=95`,`snd=generate_snd_table()`
#@markdown * 戻り値： `tuple<float, float, float>`
#@markdown * 依存関数：`find_z_from_cf_in_snd_table`
def calc_conf_interval_snd(xbar, s, n, cf=95, snd=generate_snd_table()):
  '''Calculate confidence interval of population mean of SND.

  Parameters
  ----------
  xbar: Sample mean.
  s: Population standard deviation.
  n: Sample size.
  cf: Target confidence factor. Default value is 95.
  snd: pandas DataFrame of Standard Normal Distribution Table.
    Default value is generate_snd_table function execution result

  Returns
  -------
  tuple
    lower bound, upper bound, and z-value
  '''
  import math
  z = find_z_from_cf_in_snd_table(snd, cf)[0]
  bin = z * math.sqrt(s**2 / n)
  return xbar - bin, xbar + bin, z


In [11]:
#@title #### 母比率の信頼区間の計算 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`calc_conf_interval_pp`
#@markdown * 引数：`epp`, `n`, `cf=95`,`snd=generate_snd_table()`
#@markdown * 戻り値： `tuple<float, float, float>`
#@markdown * 依存関数：`find_z_from_cf_in_snd_table`
def calc_conf_interval_pp(epp, n, cf=95, snd=generate_snd_table()):
  '''Calculate confidence interval of population proportion.

  Parameters
  ----------
  epp: Estimated population proportion.
  n: Sample size.
  cf: Target confidence factor. Default value is 95.
  snd: pandas DataFrame of Standard Normal Distribution Table.
    Default value is generate_snd_table function execution result.

  Returns
  -------
  tuple
    lower bound, upper bound, and z-value.
  '''
  import math
  z = find_z_from_cf_in_snd_table(snd, cf)[0]
  bin = z * math.sqrt((epp * (1 - epp)) / n)
  return epp - bin, epp + bin, z


In [12]:
#@title #### 母比率の区間推定における必要サンプルサイズの計算 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`calc_sample_size_with_iepp`
#@markdown * 引数：`sr`, `interval`, `cf=95`, `snd=generate_snd_table())`
#@markdown * 戻り値：`float`
#@markdown * 依存関係：`find_z_from_cf_in_snd_table`
def calc_sample_size_with_iepp(sr, interval, cf=95, snd=generate_snd_table()):
  '''Calculate required Sample Size for Interval Estimation of Population Proportion.

  Parameters
  ----------
  sr: sample ratio.
  interval: target interval estimation of population proportion.
  cf: confidence factor. default is 95.
  snd: Standard Normal Distribution Table. Default data is generated by generate_snd_table function.

  Returns
  -------
  float
    required sample size for target interval estimation of population proportion.

  '''
  import math

  z = find_z_from_cf_in_snd_table(snd, cf)[0]
  return (2 * z * math.sqrt(sr * (1 - sr)) * interval**-1) ** 2


In [13]:
#@title #### 母比率の差の信頼区間の計算{ vertical-output: true, display-mode: "form" }
#@markdown 関数名：`calc_conf_interval_unpaired`
#@markdown * 引数：`p1`, `p2`, `cf=95`,`snd=generate_td_table()`
#@markdown * 戻り値： `tuple<float, float, float, float>`
#@markdown * 依存関数：`generate_td_table`, `find_t_from_cf_in_td_table`
def calc_conf_interval_pp_diff(p1, p2, cf=95, snd=generate_snd_table()):
  '''Calculate confidence intervale of difference of population proportions.

  Parameters
  ----------
  p1: tuple of first population parameters(sample mean, unbiased estimate of variance, sample size).
  p2: tuple of second population parameters(sample mean, unbiased estimate of variance, sample size).
  cf: Target confidence factor. Default value is 95.
  snd: pandas DataFrame of Standard Normal Distribution Table.
    Default value is generate_snd_table function execution result.

  Returns
  -------
  tuple
    lower bound, upper bound, and z-value.
  '''
  import math
  z = find_z_from_cf_in_snd_table(snd, cf)[0]
  p_diff = p1[0] - p2[0]
  bin = z * math.sqrt(p1[0] * (1 - p1[0]) / p1[1] + p2[0] * (1 - p2[0]) / p2[1])
  return p_diff - bin , p_diff + bin, z


In [14]:
#@title #### 標準正規分布 $N(0, 1)$ の検定統計量 $z$ の計算 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`calc_z`
#@markdown * 引数：`phat`, `p0`, `n`
#@markdown * 戻り値：`float`
#@markdown
#@markdown $z = \dfrac{\hat{p} - p_0}{\sqrt{\dfrac{p_0 (1 - p_0)}{n}}}$
def calc_z(phat, p0, n):
  '''Calculate z-value of SND.

  Parameters
  ----------
  phat: sample proportion.
  p0: population proportion.
  n: sample size.

  Returns
  -------
  float
    z-value
  '''
  import math
  return (phat - p0) / math.sqrt(p0 * (1 - p0) / n)


In [15]:
#@title #### 二項分布 $Bi(n, p)$ の検定統計量 $z$ の計算 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`calc_z_bi`
#@markdown * 引数：`x`, `p`, `n`
#@markdown * 戻り値：`float`
#@markdown
#@markdown $z = \dfrac{X - np}{\sqrt{np(1-p)}}$
def calc_z_bi(x, p, n):
  '''Calculate z-value of binominal distribution.

  Parameters
  ----------
  x: number of successes.
  p: probability of success.
  n: number of trials.

  Returns
  -------
  float
    z-value
  '''
  import math
  return (x - n * p) / math.sqrt(n * p * (1 - p))


In [16]:
#@title #### ポアソン分布 $Po(\lambda)$ の検定統計量 $z$ の計算 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`calc_z_po`
#@markdown * 引数：`x`, `l`, `n`
#@markdown * 戻り値：`float`
#@markdown
#@markdown $z = \dfrac{X - n \lambda}{\sqrt{n\lambda}}$
def calc_z_po(x, l, n):
  '''Calculate z-value of Poisson distribution.

  Parameters
  ----------
  x: number of successes.
  l: probability of success.
  n: number of trials.

  Returns
  -------
  float
    z-value
  '''
  import math
  return (x - n * l) / math.sqrt(n * l)


In [17]:
#@title #### 母比率の差の検定統計量 $z$ の計算 { vertical-output: true, display-mode: "form" }
#@markdown 関数名：`calc_z_dpr`
#@markdown * 引数：`pp1`, `pp2`
#@markdown   * `pp1`: `p1`, `n1`
#@markdown   * `pp2`: `p2`, `n2`
#@markdown * 戻り値：`tuple<float, float>`
#@markdown
#@markdown $z =  \dfrac{\hat{p}_1 - \hat{p}_2}{\sqrt{\hat{p} (1 - \hat{p}) \left(\dfrac{1}{n_1} + \dfrac{1}{n_2}\right)}}$
#@markdown
#@markdown $where,\; \hat{p} = \dfrac{n_1 \hat{p}_1 + n_2 \hat{p}_2}{n_1 + n_2}$
def calc_z_dpr(pp1, pp2):
  '''Calculate z-value of difference in population proportion.

  Parameters
  ----------
  pp1: sample proportion and sample size of first group sample.
  pp2: sample proportion and sample size of second group sample.

  Returns
  -------
  tuple
    z-value and pooled sample proportion
  '''
  import math
  pooled_p = (pp1[0] * pp1[1] + pp2[0] * pp2[1]) / (pp1[1] + pp2[1])
  return (pp1[0] - pp2[0]) / math.sqrt(pooled_p * (1 - pooled_p) * (1 / pp1[1] + 1 / pp2[1])), pooled_p


In [18]:
# @title #### フィッシャーの $z$ 変換 { display-mode: "form" }
# @markdown 関数名：`transform_fishers_z`
# @markdown * 引数：`r`
# @markdown * 戻り値： `float`
# @markdown
# @markdown $z = \dfrac{1}{2} \log \dfrac{1 + r}{1 - r}$
def transform_fishers_z(r):
  '''Fisher's z-transform.

  Parameters
  ----------
  r: correlation coefficient of sample.

  Returns
  -------
  float
    z-value of Fisher's z-transform.
  '''
  import math
  return 1 / 2 * math.log((1 + r) / (1 - r))

In [19]:
# @title #### 母相関係数の信頼区間の計算 { vertical-output: true, display-mode: "form" }
# @markdown 関数名：`calc_conf_interval_pcc`
# @markdown * 引数：`n`, `r`, `cf=95`,`snd=generate_td_table()`
# @markdown * 戻り値： `tuple<float, float, float, float>`
# @markdown * 依存関数：`generate_td_table`, `transform_fishers_z`, `find_t_from_cf_in_td_table`
# @markdown
# @markdown $\dfrac{e^{2 z_L} - 1}{e^{2 z_L} + 1} \le \rho \le \dfrac{e^{2 z_U} - 1}{e^{2 z_U} + 1}$
# @markdown
# @markdown $where,\; $
# @markdown $z_L =  z - z_{\frac{\alpha}{2}} \sqrt{\dfrac{1}{n - 3}},\; $
# @markdown $z_U =  z + z_{\frac{\alpha}{2}} \sqrt{\dfrac{1}{n - 3}}$
def calc_conf_interval_pcc(n, r, cf=95, snd=generate_snd_table()):
  '''Calculate confidence interval of population correlation coefficient.

  Parameters
  ----------
  n: sample size.
  r: correlation coefficient of sample.
  cf: Target confidence factor. Default value is 95.
  snd: pandas DataFrame of Standard Normal Distribution Table.
    Default value is generate_snd_table function execution result.

  Returns
  -------
  tuple
    lower bound, upper bound, Fisher's z, lower z-value, and upper z-value.
  '''
  import math

  var = 1 / (n - 3)
  z = transform_fishers_z(r)
  z_a_2 = find_z_from_cf_in_snd_table(snd, cf)[0]
  z_l = z - z_a_2 * math.sqrt(var)
  z_u = z + z_a_2 * math.sqrt(var)
  lower = (math.exp(2 * z_l) - 1) / (math.exp(2 * z_l) + 1)
  upper = (math.exp(2 * z_u) - 1) / (math.exp(2 * z_u) + 1)
  return lower, upper, z, z_l, z_u


# 利用例

In [20]:
# @title #### 利用例：`generate_snd_table` { display-mode: "both" }
# @markdown 標準正規分布表の表示
snd = generate_snd_table()
snd

,0.00,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09
0.0,0.50000,0.49601,0.49202,0.48803,0.48405,0.48006,0.47608,0.47210,0.46812,0.46414
0.1,0.46017,0.45620,0.45224,0.44828,0.44433,0.44038,0.43644,0.43251,0.42858,0.42465
0.2,0.42074,0.41683,0.41294,0.40905,0.40517,0.40129,0.39743,0.39358,0.38974,0.38591
0.3,0.38209,0.37828,0.37448,0.37070,0.36693,0.36317,0.35942,0.35569,0.35197,0.34827
0.4,0.34458,0.34090,0.33724,0.33360,0.32997,0.32636,0.32276,0.31918,0.31561,0.31207
0.5,0.30854,0.30503,0.30153,0.29806,0.29460,0.29116,0.28774,0.28434,0.28096,0.27760
0.6,0.27425,0.27093,0.26763,0.26435,0.26109,0.25785,0.25463,0.25143,0.24825,0.24510
0.7,0.24196,0.23885,0.23576,0.23270,0.22965,0.22663,0.22363,0.22065,0.21770,0.21476
0.8,0.21186,0.20897,0.20611,0.20327,0.20045,0.19766,0.19489,0.19215,0.18943,0.18673
0.9,0.18406,0.18141,0.17879,0.17619,0.17361,0.17106,0.16853,0.16602,0.16354,0.16109


In [21]:
# @title #### 利用例：`generate_snd_table` { run: "auto", display-mode: "both" }
# @markdown 標準正規分布表の表示（行と列を指定）
rows = [0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.5, 2.0, 2.5, 5.0, 7.5] #@param {type:"raw"}
columns = [0.00, 0.025, 0.05, 0.075] #@param {type:"raw"}
generate_snd_table(rows=rows, columns=columns)

,0.00,0.03,0.05,0.07
0.0,0.50000,0.49003,0.48006,0.47011
0.2,0.42074,0.41099,0.40129,0.39166
0.4,0.34458,0.33542,0.32636,0.31739
0.6,0.27425,0.26599,0.25785,0.24984
0.8,0.21186,0.20469,0.19766,0.19079
1.0,0.15866,0.15268,0.14686,0.14119
1.5,0.06681,0.06363,0.06057,0.05763
2.0,0.02275,0.02143,0.02018,0.01899
2.5,0.00621,0.00578,0.00539,0.00501
5.0,0.00000,0.00000,0.00000,0.00000


In [22]:
# @title #### 利用例：`highlight_subject` { run: "auto", display-mode: "both" }
# @markdown 標準正規分布表（抜粋）で選択した行と列を強調表示
row = "0.4" #@param ["0.0", "0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9"]
column = "0.02" #@param ["0.00", "0.01", "0.02", "0.03", "0.04"]
highlight_subject(snd.iloc[:10, :5], row=row, col=column)

,0.00,0.01,0.02,0.03,0.04
0.0,0.500000,0.496010,0.492020,0.488030,0.484050
0.1,0.460170,0.456200,0.452240,0.448280,0.444330
0.2,0.420740,0.416830,0.412940,0.409050,0.405170
0.3,0.382090,0.378280,0.374480,0.370700,0.366930
0.4,0.344580,0.340900,0.337240,0.333600,0.329970
0.5,0.308540,0.305030,0.301530,0.298060,0.294600
0.6,0.274250,0.270930,0.267630,0.264350,0.261090
0.7,0.241960,0.238850,0.235760,0.232700,0.229650
0.8,0.211860,0.208970,0.206110,0.203270,0.200450
0.9,0.184060,0.181410,0.178790,0.176190,0.173610


In [23]:
# @title #### 利用例：`find_p_from_z_in_snd_table` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 標準正規分布表から指定された $z$ 値に対する確率を検索
z = 0.54 #@param {type:"number"}
p = find_p_from_z_in_snd_table(snd, z)
print('p:', p[0], ', row:', p[1], ', column:', p[2])

p: 0.2946 , row: 0.5 , column: 0.04


In [24]:
# @title #### 利用例：`highlight_z_in_snd_table` { run: "auto", display-mode: "both" }
# @markdown 標準正規分布表の指定された $z$ 値を強調表示
z = 0.54 #@param {type:"number"}
highlight_z_in_snd_table(snd, z)

,0.00,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09
0.0,0.500000,0.496010,0.492020,0.488030,0.484050,0.480060,0.476080,0.472100,0.468120,0.464140
0.1,0.460170,0.456200,0.452240,0.448280,0.444330,0.440380,0.436440,0.432510,0.428580,0.424650
0.2,0.420740,0.416830,0.412940,0.409050,0.405170,0.401290,0.397430,0.393580,0.389740,0.385910
0.3,0.382090,0.378280,0.374480,0.370700,0.366930,0.363170,0.359420,0.355690,0.351970,0.348270
0.4,0.344580,0.340900,0.337240,0.333600,0.329970,0.326360,0.322760,0.319180,0.315610,0.312070
0.5,0.308540,0.305030,0.301530,0.298060,0.294600,0.291160,0.287740,0.284340,0.280960,0.277600
0.6,0.274250,0.270930,0.267630,0.264350,0.261090,0.257850,0.254630,0.251430,0.248250,0.245100
0.7,0.241960,0.238850,0.235760,0.232700,0.229650,0.226630,0.223630,0.220650,0.217700,0.214760
0.8,0.211860,0.208970,0.206110,0.203270,0.200450,0.197660,0.194890,0.192150,0.189430,0.186730
0.9,0.184060,0.181410,0.178790,0.176190,0.173610,0.171060,0.168530,0.166020,0.163540,0.161090


In [25]:
# @title #### 利用例：`find_z_from_p_in_snd_table` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 標準正規分布表から指定された確率に対する $z$ 値を検索
# @markdown
# @markdown 確率
p = 0.23 #@param {type:"number"}
z = find_z_from_p_in_snd_table(snd, p)
print('z:', z[0], ', row:', z[1], ', column:', z[2])

z: 0.74 , row: 0.7 , column: 0.04


In [26]:
# @title #### 利用例：`highlight_probability_in_snd_table` { run: "auto", display-mode: "both" }
# @markdown 標準正規分布表の指定された確率に対する $z$ 値を強調表示
# @markdown
# @markdown 確率
p = 0.23 #@param {type:"number"}
highlight_probability_in_snd_table(snd, p)

,0.00,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09
0.0,0.500000,0.496010,0.492020,0.488030,0.484050,0.480060,0.476080,0.472100,0.468120,0.464140
0.1,0.460170,0.456200,0.452240,0.448280,0.444330,0.440380,0.436440,0.432510,0.428580,0.424650
0.2,0.420740,0.416830,0.412940,0.409050,0.405170,0.401290,0.397430,0.393580,0.389740,0.385910
0.3,0.382090,0.378280,0.374480,0.370700,0.366930,0.363170,0.359420,0.355690,0.351970,0.348270
0.4,0.344580,0.340900,0.337240,0.333600,0.329970,0.326360,0.322760,0.319180,0.315610,0.312070
0.5,0.308540,0.305030,0.301530,0.298060,0.294600,0.291160,0.287740,0.284340,0.280960,0.277600
0.6,0.274250,0.270930,0.267630,0.264350,0.261090,0.257850,0.254630,0.251430,0.248250,0.245100
0.7,0.241960,0.238850,0.235760,0.232700,0.229650,0.226630,0.223630,0.220650,0.217700,0.214760
0.8,0.211860,0.208970,0.206110,0.203270,0.200450,0.197660,0.194890,0.192150,0.189430,0.186730
0.9,0.184060,0.181410,0.178790,0.176190,0.173610,0.171060,0.168530,0.166020,0.163540,0.161090


In [27]:
# @title #### 利用例：`find_z_from_cf_in_snd_table` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 標準正規分布表から指定された信頼係数に対する $z$ 値を検索
# @markdown
# @markdown 信頼係数
cf = 95 #@param ["90", "95", "99"] {type:"raw"}
z = find_z_from_cf_in_snd_table(snd, cf)
print('z:', z[0], ', p:', z[1], ', row:', z[2], ', column:', z[3])

z: 1.96 , p: 0.025 , row: 1.9 , column: 0.06


In [28]:
# @title #### 利用例：`calc_conf_interval_snd` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 母平均の信頼区間の計算
# @markdown
# @markdown 標本平均、不偏分散、サンプルサイズ、信頼係数
xbar = 172 #@param {type:"number"}
s = 5.5 #@param {type:"number"}
n = 100 #@param {type:"integer"}
cf = 95 #@param ["90", "95", "99"] {type:"raw", allow-input: true}
ci = calc_conf_interval_snd(xbar, s, n, cf)
print('lower:', ci[0], ', upper:', ci[1], ', z:', round(ci[2], 2))

lower: 170.922 , upper: 173.078 , z: 1.96


In [29]:
# @title #### 利用例：`calc_conf_interval_pp` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 母比率の信頼区間の計算
# @markdown
# @markdown 母比率、サンプルサイズ、信頼係数
epp = 0.15 #@param {type:"number"}
n = 100 #@param {type:"integer"}
cf = 95 #@param ["90", "95", "99"] {type:"raw", allow-input: true}
ci = calc_conf_interval_pp(epp, n, cf)
print('lower:', round(ci[0], 5), ', upper:', round(ci[1], 5), ', z:', round(ci[2], 2))

lower: 0.08001 , upper: 0.21999 , z: 1.96


In [30]:
# @title #### 利用例：`calc_sample_size_with_iepp` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 母比率の区間推定における必要サンプルサイズの計算
# @markdown
# @markdown 標本比率、ターゲットとなる母比率の区間推定、信頼係数
sr = 0.1 #@param {type:"number"}
interval = 0.05 #@param {type:"number"}
cf = 95 #@param ["90", "95", "99"] {type:"raw"}
round(calc_sample_size_with_iepp(sr, interval, cf), 1)


553.2

In [31]:
# @title #### 利用例：`calc_conf_interval_pp_diff` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 母比率の差の信頼区間の計算
# @markdown
# @markdown 母集団1の統計量（母比率、サンプルサイズ）
epp1 = 0.4 #@param {type:"number"}
n1 = 200 #@param {type:"integer"}
# @markdown 母集団2の統計量（母比率、サンプルサイズ）
epp2 = 0.2 #@param {type:"number"}
n2 = 300 #@param {type:"integer"}
# @markdown 信頼係数
cf = 95 #@param ["90", "95", "99"] {type:"raw", allow-input: true}
ci = calc_conf_interval_pp_diff((epp1, n1), (epp2, n2), cf)
print('lower:', round(ci[0], 5),
      ', upper:', round(ci[1], 5),
      ', z:', round(ci[2], 2))

lower: 0.1184 , upper: 0.2816 , z: 1.96


In [32]:
# @title #### 利用例：`calc_z` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 標準正規分布 $N(0, 1)$ の検定統計量 $z$ の計算（比率が小数）
# @markdown
# @markdown 標本比率、母比率、サンプルサイズ
phat = 0.18333 #@param {type:"number"}
p0 = 0.16667 #@param {type:"number"}
n = 12000 #@param {type:"integer"}
round(calc_z(phat, p0, n), 5)

4.89698

In [33]:
# @title #### 利用例：`calc_z` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 標準正規分布 $N(0, 1)$ の検定統計量 $z$ の計算（比率が分数）
# @markdown
# @markdown 標本比率（分数）
phat_n = 11 #@param {type:"integer"}
phat_d = 60 #@param {type:"integer"}
# @markdown 母比率（分数）
p0_n = 1 #@param {type:"integer"}
p0_d = 6 #@param {type:"integer"}
# @markdown サンプルサイズ
n = 12000 #@param {type:"integer"}
print('phat:', round(phat_n / phat_d, 5))
print('p0  :', round(p0_n / p0_d, 5))
round(calc_z(phat_n / phat_d, p0_n / p0_d, n), 5)

phat: 0.18333
p0  : 0.16667


4.89898

In [34]:
# @title #### 利用例：`calc_z_bi` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 二項分布 $Bi(n, p)$ の検定統計量 $z$ の計算
# @markdown
# @markdown 成功回数、試行回数、成功確率
x = 2200 #@param {type:"number"}
n = 12000 #@param {type:"integer"}
p = 0.16667 #@param {type:"number"}
round(calc_z_bi(x, p, n), 5)

4.89796

In [35]:
# @title #### 利用例：`calc_z_po` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown ポアソン分布 $Po(\lambda)$ の検定統計量 $z$ の計算
# @markdown
# @markdown 発生回数、$\lambda$、サンプルサイズ
x = 200 #@param {type:"number"}
l = 20 #@param {type:"number"}
n = 12 #@param {type:"integer"}
round(calc_z_po(x, l, n), 5)

-2.58199

In [36]:
# @title #### 利用例：`calc_z_dpr` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 母比率の差の検定統計量 $z$ の計算
# @markdown
# @markdown 母集団1の統計量（母比率、サンプルサイズ）
p1 = 0.2 #@param {type:"number"}
n1 = 5000 #@param {type:"integer"}
# @markdown 母集団2の統計量（母比率、サンプルサイズ）
p2 = 0.18 #@param {type:"number"}
n2 = 3000 #@param {type:"integer"}
z = calc_z_dpr((p1, n1), (p2, n2))
print('z:', round(z[0], 5), ', pooled p:', round(z[1], 5))

z: 2.19656 , pooled p: 0.1925


In [37]:
# @title #### 利用例：`transform_fishers_z` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown フィッシャーの $z$ 変換
# @markdown
# @markdown 標本の相関係数
r = 0.50 #@param {type:"number"}
round(transform_fishers_z(r), 5)

0.54931

In [38]:
# @title #### 利用例：`calc_conf_interval_pcc` { run: "auto", vertical-output: true, display-mode: "both" }
# @markdown 母相関係数の信頼区間の計算
# @markdown
# @markdown サンプルサイズ、標本の相関係数、信頼係数
n = 200 #@param {type:"integer"}
r = 0.50 #@param {type:"number"}
cf = 95 #@param ["90", "95", "99"] {type:"raw", allow-input: true}

ci_21_2_2 = calc_conf_interval_pcc(n, r, cf)
print('lower:', round(ci_21_2_2[0], 5),
      ', upper:', round(ci_21_2_2[1], 5),
      ', Fisher\'s z:', round(ci_21_2_2[1], 5),
      ', lower z:', round(ci_21_2_2[1], 5),
      ', upper z:', round(ci_21_2_2[2], 2))

lower: 0.38819 , upper: 0.59731 , Fisher's z: 0.59731 , lower z: 0.59731 , upper z: 0.55
